## Load project

In [1]:
from os import path, getenv
from mlrun import load_project, mount_v3io
import nuclio

In [2]:
project_name = '-'.join(filter(None, ['rapid-prototype', getenv('V3IO_USERNAME', None)]))
project_path = path.abspath('conf')
project = load_project(name=project_name, context=project_path, init_git=False)

print(f'Project path: {project_path}\nProject name: {project_name}')

Project path: /User/rapid-proto/conf
Project name: rapid-prototype-michaelk


### Add Serving Functions from the MLRun Functions Marketplace <a id="gs-ml-pipeline-add-functions"></a>


In [3]:
project.to_dict()

{'name': 'rapid-prototype-michaelk',
 'functions': [{'name': 'snapshot-data',
   'spec': {'kind': 'job',
    'metadata': {'name': 'snapshot-data',
     'tag': '',
     'project': 'rapid-prototype-michaelk'},
    'spec': {'command': '',
     'args': [],
     'image': '.mlrun/func-rapid-prototype-michaelk-snapshot-data-latest',
     'env': [{'name': 'V3IO_API', 'value': ''},
      {'name': 'V3IO_USERNAME', 'value': ''},
      {'name': 'V3IO_ACCESS_KEY', 'value': ''}],
     'default_handler': '',
     'entry_points': {'snapshot_data': {'name': 'snapshot_data',
       'doc': '',
       'parameters': [{'name': 'context', 'default': ''},
        {'name': 'container', 'default': ''},
        {'name': 'table_path', 'default': ''},
        {'name': 'columns', 'default': ''},
        {'name': 'format', 'default': 'csv'}],
       'outputs': [{'default': ''}],
       'lineno': 8}},
     'description': '',
     'build': {'functionSourceCode': 'IyBHZW5lcmF0ZWQgYnkgbnVjbGlvLmV4cG9ydC5OdWNsaW9FeHBvcnR

In [6]:
project.set_function('hub://model_server:development', 'serving')

input_stream = f'http://v3io-webapi:8081/users/{getenv("V3IO_USERNAME")}/examples/rapid-prototype/serving-stream@modelserver'
partitions = list(range(0,8))

serving = project.func('serving').apply(mount_v3io())
serving.add_model('rapid_proto', '/User/rapid-proto/artifacts/model/3/model.pkl')
serving.set_envs({'INFERENCE_STREAM' : path.join('users', getenv('V3IO_USERNAME'), 'examples/rapid-prototype/inference-stream')})

serving.add_trigger('serving_stream', nuclio.triggers.V3IOStreamTrigger(url=input_stream, partitions=partitions))
serving.spec.config.pop('spec.triggers.http')

serving.spec.min_replicas= 1
serving.spec.to_dict()

{'command': '',
 'args': [],
 'image': '',
 'description': 'generic sklearn model server',
 'min_replicas': 1,
 'max_replicas': 4,
 'volumes': [{'flexVolume': {'driver': 'v3io/fuse',
    'options': {'accessKey': 'b01eb2f1-294a-4f63-b0a6-42561e3e1706',
     'container': 'users',
     'subPath': '/michaelk'}},
   'name': 'v3io'}],
 'volume_mounts': [{'mountPath': '/User', 'name': 'v3io'}],
 'env': [{'name': 'MODEL_CLASS', 'value': 'ClassifierModel'},
  {'name': 'ENABLE_EXPLAINER', 'value': 'False'},
  {'name': 'V3IO_API', 'value': 'v3io-webapi.default-tenant.svc:8081'},
  {'name': 'V3IO_USERNAME', 'value': 'michaelk'},
  {'name': 'V3IO_ACCESS_KEY', 'value': 'b01eb2f1-294a-4f63-b0a6-42561e3e1706'},
  {'name': 'SERVING_MODEL_rapid_proto',
   'value': '/User/rapid-proto/artifacts/model/3/model.pkl',
   'value_from': None},
  {'name': 'INFERENCE_STREAM',
   'value': 'users/michaelk/examples/rapid-prototype/inference-stream',
   'value_from': None}],
 'config': {'spec.triggers.serving_stream'

In [8]:
serving.deploy(project='rapid-prototype-mk')

> 2020-07-30 15:10:08,452 [info] deploy started
[nuclio] 2020-07-30 15:10:10,566 (info) Build complete
[nuclio] 2020-07-30 15:10:23,728 (info) Function deploy complete
[nuclio] 2020-07-30 15:10:23,736 done creating rapid-prototype-mk-sklearn-server, function address: 192.168.226.12:30693


'http://192.168.226.12:30693'

### Trigger the seving func

In [54]:
import requests
import json

feat = [7,799,47,17,143,1560,7,164,11,810.0,162.0,5.0,782.0]
json = json.dumps({'instances': np.array(feat).reshape(1,-1).tolist()})

resp = requests.post(url='http://192.168.226.12:31425/rapid_proto/predict', 
                     json = json)
print(resp.status_code)
print(resp.text)

200
[0]
